# 準備

## 基本モジュール

In [72]:
from pprint import pprint
from datetime import date
import pandas as pd

## スクレイピング

In [2]:
# スクレイピング関連
from time import sleep
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

---

# 分析準備

In [38]:
options = webdriver.ChromeOptions()
# Chrome を立ち上げて確認したい場合は次の行をコメント化
options.add_argument('--headless')

# ブラウザの立ち上げ
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
url = "https://www.dell.com/ja-jp/shop?~ck=bt"
browser.get(url)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\kny16\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache
C:\Users\kny16\AppData\Local\Temp\ipykernel_16912\298012747.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [39]:
pc_dict = {"デスクトップパソコン": "Desktop", "ノートパソコンと2-in-1 PC": "Laptop", "ゲーミングデスクトップ": "Gaming Desktop", "ゲーミングノート": "Gaming Laptop", "モニター・アクセサリー・その他周辺機器": "Peripheral"}

In [73]:
today = date.today()
today = str(today)

In [45]:
def get_pc_data(pc_prod, target):
    pc_data_list = list()

    for pc in pc_prod:
        pc_name = pc.find_element(By.CLASS_NAME, "ps-title")
        pc_price = pc.find_element(By.CLASS_NAME, "ps-dell-price.ps-simplified")
        pc_pros = pc.find_element(By.CLASS_NAME, "short-specs.ps-dds-font-icon.dds_processor")
        pc_win = pc.find_element(By.CLASS_NAME, "short-specs.ps-dds-font-icon.dds_disc-system")
        pc_gpu = pc.find_element(By.CLASS_NAME, "short-specs.ps-dds-font-icon.dds_video-card")
        pc_mem = pc.find_element(By.CLASS_NAME, "short-specs.ps-dds-font-icon.dds_memory")
        pc_hdd = pc.find_element(By.CLASS_NAME, "short-specs.ps-dds-font-icon.dds_hard-drive")
        pc_camp = pc.find_elements(By.CLASS_NAME, "ps-special-offers-expanded-link")


        # リストに保存
        pc_data = target.split("_")
        pc_data.append(pc_name.text.replace("\n", ""))
        pc_data.append(pc_price.text.replace("\n", "").replace("販売価格", ""))
        if pc_gpu != None:
            pc_data.append(pc_pros.text.replace("\n", "").replace(" ", ""))
        else:
            pc_data.append("-")

        if pc_win != None:
            pc_data.append(pc_win.text.replace("\n", "").replace(" ", ""))
        else:
            pc_data.append("-")

        if pc_gpu != None:
            pc_data.append(pc_gpu.text.replace("\n", "").replace(" ", ""))
        else:
            pc_data.append("-")

        if pc_gpu != None:
            pc_data.append(pc_mem.text.replace("\n", "").replace(" ", ""))
        else:
            pc_data.append("-")

        if pc_hdd != None:
            pc_data.append(pc_hdd.text.replace("\n", "").replace(" ", ""))
        else:
            pc_data.append("-")

        for i in range(len(pc_camp)):
            pc_data.append(pc_camp[i].text.replace("\n", "").replace(" ", "").replace("\u200b", ""))

        pc_data_list.append(pc_data)
    return pc_data_list


# 取得開始

In [40]:
pc_class_list = list()
pc_href_list = list()

WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "category-text-margin.col-lg-6.col-md-6.col-sm-6")))
pc_type = browser.find_elements(By.CLASS_NAME, "category-text-margin.col-lg-6.col-md-6.col-sm-6")
for i in range(1, 6):
    # TOPよりPCの種類を取得
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, "h5")))
    pc_types = pc_type[i].find_element(By.TAG_NAME, "h5")
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "sub-category.unstyled")))
    pc_names = pc_type[i].find_elements(By.CLASS_NAME, "sub-category.unstyled")
    pc_name = pc_names[0].text.split("\n")

    # 各製品への遷移先のURLの取得
    pc_href = pc_names[0].find_elements(By.TAG_NAME, "a")

    pc_url_list = list()
    pc_type_list = list()

    # リストに保存
    for pc in range(len(pc_name)):
        target = pc_dict[pc_types.text] + "_" + pc_name[pc].replace("ノート・2-in-1", "")
        pc_class_list.append(target)
        pc_url = pc_href[pc].get_attribute('href')
        pc_href_list.append(pc_url)

pc_class_list = pc_class_list[:-1]
pc_href_list = pc_href_list[:-1]

In [54]:
print("【要素の取得中】")
pc_data_list = list()

for i in range(len(pc_href_list)):
    target = pc_class_list[i]
    browser.get(pc_href_list[i])
    print("\n～ " + target + " ～")

    k = 1
    while True:
        print(f"{k}ページ目")
        
        pc_prod = browser.find_elements(By.CLASS_NAME, "stack-system.ps-stack")
        pc_data = get_pc_data(pc_prod, target)
        pc_data_list.extend(pc_data)
        
        # 次のページへ
        next_page = browser.find_elements(By.CLASS_NAME, "system-result-btn.page.pagination-btn.prev-next-btn")

        # 次のぺージが無い場合
        if next_page == []:
            break
        if next_page[1].get_attribute('data-disabled') == "True":
            break

        next_page[1].click()
        k += 1

print("\n【取得完了】")

# ブラウザを閉じる
browser.close()

【要素の取得中】

～ Laptop_XPSノート ～
1ページ目
2ページ目
3ページ目

～ Laptop_Inspiron ～
1ページ目
2ページ目
3ページ目
4ページ目
5ページ目
6ページ目
7ページ目
8ページ目
9ページ目
10ページ目
11ページ目
12ページ目
13ページ目
14ページ目
15ページ目
16ページ目
17ページ目

～ Laptop_Alienware ～
1ページ目
2ページ目
3ページ目

～ Laptop_Dell G ～
1ページ目
2ページ目
3ページ目
4ページ目

～ Desktop_XPS ～
1ページ目
2ページ目

～ Desktop_Inspiron ～
1ページ目
2ページ目
3ページ目
4ページ目

～ Desktop_Alienware ～
1ページ目
2ページ目

～ Desktop_Vostro ～
1ページ目
2ページ目
3ページ目
4ページ目

～ Gaming Laptop_Alienware ～
1ページ目
2ページ目
3ページ目

～ Gaming Laptop_Dell G ～
1ページ目
2ページ目
3ページ目
4ページ目

～ Gaming Desktop_Alienware ～
1ページ目
2ページ目

～ Peripheral_モニター・モニターアクセサリー ～
1ページ目

～ Peripheral_PCアクセサリー ～
1ページ目

～ Peripheral_ゲーミングモニター・ゲーミングアクセサリー ～
1ページ目

～ Peripheral_プロジェクタ＆プロジェクタアクセサリ ～
1ページ目

【取得完了】


In [66]:
# データフレーム化してcsvに保存
df_pc = pd.DataFrame(pc_data_list, columns=["分類", "種類", "品名", "販売価格", "CPU", "OS", "GPU", "RAM", "SSD/HDD", "クーポン1", "クーポン2"])
display(df_pc.head())

df_pc.to_csv(f"Dell_PC_{today}.csv", encoding="utf-8", index=False)

,分類,種類,品名,販売価格,CPU,OS,GPU,RAM,SSD/HDD,クーポン1,クーポン2
0,Laptop,XPSノート,XPS 13 プレミアム（シルバー）,"104,784円",第11世代インテル®Core™i5-1135G7プロセッサー,Windows11Home,インテル®Iris®Xeグラフィックス共有グラフィックスメモリー付き,8GB4267MHzLPDDR4xメモリーオンボード,256GBM.2PCIeNVMeSSD,20%OFFクーポン適用済:EFR23O20,"＜マカフィー®リブセーフ™ご購入キャンペーン＞Amazonギフト券6,000円分が抽選で105名"
1,Laptop,XPSノート,XPS 13 プレミアム（大容量メモリー・シルバー）,"122,863円",第11世代インテル®Core™i5-1135G7プロセッサー,Windows11Home,インテル®Iris®Xeグラフィックス共有グラフィックスメモリー付き,16GB4267MHzLPDDR4xメモリーオンボード,"512GB,M.2,SSD,Class35",20%OFFクーポン適用済:EFR23O20,"＜マカフィー®リブセーフ™ご購入キャンペーン＞Amazonギフト券6,000円分が抽選で105名"
2,Laptop,XPSノート,XPS 13 プレミアム（Office H&B・シルバー）,"125,024円",第11世代インテル®Core™i5-1135G7プロセッサー,Windows11Home,インテル®Iris®Xeグラフィックス共有グラフィックスメモリー付き,8GB4267MHzLPDDR4xメモリーオンボード,256GBM.2PCIeNVMeSSD,20%OFFクーポン適用済:EFR23O20,"＜マカフィー®リブセーフ™ご購入キャンペーン＞Amazonギフト券6,000円分が抽選で105名"
3,Laptop,XPSノート,XPS 13 【新生活応援・即納】プレミアム（Office H&B・シルバー・ヘッドセット付）,"130,194円",第11世代インテル®Core™i5-1135G7プロセッサー,Windows11Home,インテル®Iris®Xeグラフィックス共有グラフィックスメモリー付き,8GB4267MHzLPDDR4xメモリーオンボード,256GBM.2PCIeNVMeSSD,20%OFFクーポン適用済:EFR23O20,"＜マカフィー®リブセーフ™ご購入キャンペーン＞Amazonギフト券6,000円分が抽選で105名"
4,Laptop,XPSノート,XPS 13 プレミアム（大容量メモリー・Office H&B・シルバー）,"143,103円",第11世代インテル®Core™i5-1135G7プロセッサー,Windows11Home,インテル®Iris®Xeグラフィックス共有グラフィックスメモリー付き,16GB4267MHzLPDDR4xメモリーオンボード,"512GB,M.2,SSD,Class35",20%OFFクーポン適用済:EFR23O20,"＜マカフィー®リブセーフ™ご購入キャンペーン＞Amazonギフト券6,000円分が抽選で105名"
